In [41]:
import gurobipy as gp
from gurobipy import GRB

In [42]:
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd

In [43]:
# 1. Chargement des données
data = pd.read_csv("data/breastcancer_processed.csv")
# X = data.data
# y = data.target
# feature_names = data.feature_names

In [44]:
data.head()
X, y, feature_names = data.drop(columns="Benign"), data["Benign"], data.columns

In [45]:
# 2. Entraînement du modèle pour obtenir les poids (coefficients)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

model = LogisticRegression(max_iter=10000)
model.fit(X_scaled, y)

poids_cancer = model.coef_[0] 

In [46]:
# 3. Sélection de deux patients à comparer
# On cherche un patient 'Malignant' (0) et un 'Benign' (1)
idx_malignant = np.where(y == 0)[0][0]
idx_benign = np.where(y == 1)[0][0]

patient_M = X_scaled[idx_malignant]
patient_B = X_scaled[idx_benign]

In [47]:
# 4. Adaptation des fonctions utilitaires pour ce dataset
def preparer_donnees_cancer(notes_a, notes_b, poids):
    """Calcule les contributions pour le dataset Breast Cancer."""
    # Contribution = poids * différence de note
    contribs = [poids[i] * (notes_a[i] - notes_b[i]) for i in range(len(poids))]
    P = [i for i, c in enumerate(contribs) if c > 0]
    C = [i for i, c in enumerate(contribs) if c < 0]
    return contribs, P, C

In [48]:
def afficher_resultats(titre, status, explication):
    """Affiche proprement le résultat d'un modèle d'optimisation."""
    print(f"\n=== {titre} ===")
    if status == "OPTIMAL" and explication:
        print("SUCCÈS : Une explication a été trouvée :")
        for groupe in explication:
            print(f"  • {groupe}")
    else:
        print("ÉCHEC : Aucune explication de ce type n'est possible pour ces candidats.")
    print("-" * 50)

In [63]:
def explication_hybride_cancer(notes_a, notes_b, poids, noms_features):
    """Version adaptée de l'explication hybride pour les données médicales."""
    # Calcul des contributions
    contribs, P, C = preparer_donnees_cancer(notes_a, notes_b, poids)

    # Modèle
    m = gp.Model("Cancer_Explanation")
    m.setParam('OutputFlag', 0)

    # Variables de liaison pour les deux types de structures
    x_1m = {} # x_1m[i,j] = 1 si le Pro i (Hub) couvre le Con j
    x_m1 = {} # x_m1[i,j] = 1 si le Con j (Hub) est couvert par le Pro i

    for i in P:
        for j in C:
            x_1m[i, j] = m.addVar(vtype=GRB.BINARY, name=f"1m_{noms_features[i]}_{noms_features[j]}")
            x_m1[i, j] = m.addVar(vtype=GRB.BINARY, name=f"m1_{noms_features[i]}_{noms_features[j]}")

    # Indicateurs de rôle "Hub"
    # h_P[i] = 1 si le Pro i est utilisé comme centre d'un (1-m)
    h_P = {i: m.addVar(vtype=GRB.BINARY) for i in P}
    
    # h_C[j] = 1 si le Con j est utilisé comme centre d'un (m-1)
    h_C = {j: m.addVar(vtype=GRB.BINARY) for j in C}

    # Contraintes
    # Contrainte C1: Tout Con doit être couvert exactement une fois
    for j in C:
        m.addConstr(gp.quicksum(x_1m[i, j] for i in P) + h_C[j] == 1, name=f"Cover_{j}")

    # Contrainte C2: Tout Pro utilisé au max une fois
    for i in P:
        m.addConstr(h_P[i] + gp.quicksum(x_m1[i, j] for j in C) <= 1, name=f"Use_{i}")

    # Contrainte C3: Cohérence logique des liens
    for i in P:
        for j in C:
            m.addConstr(x_1m[i, j] <= h_P[i], name=f"Logic_1m_{i}_{j}")
            m.addConstr(x_m1[i, j] <= h_C[j], name=f"Logic_m1_{i}_{j}")

    # Contrainte C4: Validité des Trade-offs (Somme pondérée > 0)
    
    for i in P: # Pour chaque Pro Hub (Structure 1-m): delta_i + sum(delta_j * x_1m_ij) >= 0
        contrib_cons = gp.quicksum(contribs[j] * x_1m[i, j] for j in C)
        m.addConstr(contribs[i] * h_P[i] + contrib_cons >= 0, name=f"Valid_1m_{i}")
    
    for j in C: # Pour chaque Con Hub (Structure m-1): sum(delta_i * x_m1_ij) + delta_j >= 0
        contrib_pros = gp.quicksum(contribs[i] * x_m1[i, j] for i in P)
        m.addConstr(contrib_pros + contribs[j] * h_C[j] >= 0, name=f"Valid_m1_{j}")

    # Résolution
    m.optimize()

    # Analyse
    if m.Status == GRB.OPTIMAL:
        res = []
        # Structures (1-m)
        for i in P:
            if h_P[i].X > 0.5:
                cons_covered = [noms_features[j] for j in C if x_1m[i, j].X > 0.5]
                if cons_covered:
                    res.append(f"Groupe (1-m) : L'avantage {noms_features[i]} (+{contribs[i]}) compense {cons_covered}")
        
        # Structures (m-1)
        for j in C:
            if h_C[j].X > 0.5:
                pros_used = [noms_features[i] for i in P if x_m1[i, j].X > 0.5]
                val_pros = sum(contribs[i] for i in P if x_m1[i, j].X > 0.5)
                res.append(f"Groupe (m-1) : Les avantages {pros_used} (+{val_pros}) compensent {noms_features[j]} ({contribs[j]})")
        
        return "OPTIMAL", res
    
    return "INFEASIBLE", None


# 5. Exécution
status, explication = explication_hybride_cancer(patient_M, patient_B, poids_cancer, feature_names)
afficher_resultats("Analyse du Cancer du Sein : Patient Malin vs Bénin", status, explication)


=== Analyse du Cancer du Sein : Patient Malin vs Bénin ===
ÉCHEC : Aucune explication de ce type n'est possible pour ces candidats.
--------------------------------------------------
